In [9]:
#from rr.nartd import extract_file as ef
import glob
import logging
import tempfile
import zipfile
import py7zr
import re

from rr.nartd.store_cleaner import StoreCleaner
from rube.data.clean import RawDataCleaner
import polars as pl
import pandas as pd
from rr.nartd.first_set import import_data
from rr.nartd.store_cleaner import StoreCleaner
from rr.nartd.tokenizers import TokenizerFactory
from rube.data.tokenize import serialize_tokenizer, deserialize_tokenizer
from rube.data.real import ObservationalGenerator

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
dsetde = import_data(n_lines = 10000000, country='de', zip_pathname='/mnt/g/My Drive/Python/Kantar data/Original zip files/Germany-230208_EP Coke dissag data_Jan 2021 - Dec 2022.7z')
#dsetuk = import_data(n_lines = 10000000, country='uk', zip_pathname='/mnt/g/My Drive/Python/Kantar data/Original zip files/HHP GB DATA.zip')
#dsetfr = import_data(n_lines = 10000000, country='fr', zip_pathname='/mnt/g/My Drive/Python/Kantar data/Original zip files/France- Disaggregated data - COCA - 2020P1 A 2022P13.zip')


In [17]:
cl = StoreCleaner(period_in_weeks = 6,
                  min_visits = 30,
                  min_baskets = 180,
                  min_average_spend = 0,
                  max_accepted_quantity = 12,
                  remove_singleton_baskets= False,
                  variety_quantile_bound=0.999)


In [20]:
dset = cl.transform_data(dsetde)

100%|██████████| 105/105 [01:05<00:00,  1.60it/s]
/home/paulmainwood/projects/RUBE_proprietary/lib/rube/data/clean.py:206: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  f"${gb_i.Spend.sum().mean():2.2f} with {int(counts.mean())} items.")
/home/paulmainwood/projects/RUBE_proprietary/lib/rube/data/clean.py:208: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  f"The biggest contains {int(counts.max())} items.")


In [21]:
dset

,Customer ID,StockCode,Invoice,Quantity,Spend,Price,Week,Period,Variety,invoice_token
0,000013feca465b03def897639b5c2dcd,COCA-COLA|500ml|PET Not returnable|COLA|NARTD ...,000013feca465b03def897639b5c2dcd|KAUFLAND|95,12,11.88,0.990,95,15|KAUFLAND,3,0
1,000013feca465b03def897639b5c2dcd,FANTA|330ml|Can|ERDBEER-MISCHUNG|NARTD Regular...,000013feca465b03def897639b5c2dcd|KAUFLAND|95,2,1.70,0.850,95,15|KAUFLAND,3,0
2,000013feca465b03def897639b5c2dcd,SURPRISE|300ml|PET Not returnable|MULTIVITAMIN...,000013feca465b03def897639b5c2dcd|KAUFLAND|95,12,14.28,1.190,95,15|KAUFLAND,3,0
3,000013feca465b03def897639b5c2dcd,PL BONEAU|1500ml|PET Not returnable|ROTE FRUEC...,000013feca465b03def897639b5c2dcd|LIDL|1,12,6.84,0.570,1,0|LIDL,1,1
4,000013feca465b03def897639b5c2dcd,PL FREEWAY|1500ml|PET Not returnable|APFEL / A...,000013feca465b03def897639b5c2dcd|LIDL|11,12,9.48,0.790,11,1|LIDL,2,2
...,...,...,...,...,...,...,...,...,...,...
939112,fff5fac94e2727efb2dd1b204b53f7f0,PL SOLEVITA|1000ml|Carton|JOHANNISBEERE/CASSIS...,fff5fac94e2727efb2dd1b204b53f7f0|LIDL|76,2,1.78,0.890,76,12|LIDL,1,504133
939113,fff5fac94e2727efb2dd1b204b53f7f0,PL SOLEVITA|1000ml|Carton|JOHANNISBEERE/CASSIS...,fff5fac94e2727efb2dd1b204b53f7f0|LIDL|82,4,3.56,0.890,82,13|LIDL,1,504134
939114,fff5fac94e2727efb2dd1b204b53f7f0,PL SOLEVITA|1000ml|Carton|JOHANNISBEERE/CASSIS...,fff5fac94e2727efb2dd1b204b53f7f0|LIDL|86,4,3.58,0.895,86,14|LIDL,1,504135
939115,fff5fac94e2727efb2dd1b204b53f7f0,PL SOLEVITA|1000ml|Carton|JOHANNISBEERE/CASSIS...,fff5fac94e2727efb2dd1b204b53f7f0|LIDL|91,3,2.85,0.950,91,15|LIDL,1,504136


In [10]:
#Import and clean the data:
country_code = 'uk'

#Load and apply the appropriate Tokenizer for the data:
tokenizer = TokenizerFactory.create(
    country_code=country_code,
    stock_vocab_size=20001,
    user_vocab_size=20001,
    user_clusters=0
)

In [11]:
tokenizer.fit(dset)

/home/paulmainwood/projects/RUBE_proprietary/lib/rube/data/tokenize.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['user_cluster'] = user_cluster


In [12]:
tokenized_data = tokenizer.transform(dset)

In [15]:
tokenized_data['scans']

,product_token,period_token,user_token,Price,Quantity,Spend
invoice_token,,,,,,
0,88,136,6,1.59,1,1.59
1,449,105,6,1.40,2,2.80
2,14,107,6,0.95,1,0.95
2,55,107,6,0.33,2,0.66
2,73,107,6,0.47,1,0.47
...,...,...,...,...,...,...
940560,467,76,1,2.00,1,2.00
940561,981,76,1,0.70,1,0.70
940561,982,76,1,1.50,2,3.00


In [9]:
tokenizer.set_outside_prices(tokenized_data['scans'])

In [37]:
serialize_tokenizer(tokenizer, fit_dir / 'tokenizer')

NameError: name 'fit_dir' is not defined

In [ ]:
dsetuk_pandas_sorted['Price'].sum()

np.float64(3711944.304529911)

In [ ]:
dsetuk_polars_sorted['Price'].sum()

3711944.3069365886

In [17]:
dff = data_cleaned.to_pandas()[['StockCode', 'Quantity', 'Spend']]
minimal_tokens = dff.groupby(['StockCode']).sum().sort_values(ascending=False, by='Spend')
minimal_tokens['share'] = minimal_tokens.Spend / minimal_tokens.Spend.sum()
tokenizer.token_shares = minimal_tokens.reset_index()

In [18]:
tokenizer.token_shares

,StockCode,Quantity,Spend,share
0,COCA-COLA|1250ml|PET Not returnable|COLA|NARTD...,95992,104180.176071,0.027507
1,COCA-COLA|1000ml|PET Returnable|COLA|NARTD Reg...,63109,67219.839316,0.017748
2,PL SASKIA|1500ml|PET Not returnable|<undefined...,242333,66281.050544,0.017500
3,COCA-COLA ZERO|1250ml|PET Not returnable|COLA|...,55506,62606.494876,0.016530
4,PL NATURALIS|1500ml|PET Not returnable|<undefi...,160449,44247.459703,0.011683
...,...,...,...,...
1412,PL SOLEVITA|500ml|PET Not returnable|MANGO|NAR...,429,209.170003,0.000055
1413,PL GENUSS PLUS|500ml|PET Not returnable|<undef...,417,203.160004,0.000054
1414,PL GENUSS PLUS|500ml|PET Not returnable|MULTIV...,232,160.529999,0.000042
1415,PL SOLEVITA|200ml|Carton|EXOTISCHE / TROPISCHE...,446,126.499999,0.000033


In [18]:
split_data = tokenizer.token_shares.StockCode.str.split('|')

In [19]:
split_data

0       [Pepsi Max S/Fr Cola, 7920, Cans, Cola, Diet, ...
1       [Pepsi Max S/Fr Cola, 2000, Plastic Bottle, Co...
2       [Diet Coke, 7920, Cans, Cola, Diet, Carbonated...
3       [Tesco Dbl Stng NAS Squash, 15000, Plastic Bot...
4       [Coca Cola, 7920, Cans, Cola, Regular, Carbona...
                              ...                        
1380    [Pure Life Sprng Mnrl Wtr, 1500, Plastic Bottl...
1381    [Old Jamaica Reg, 330, Plastic Bottle, Other F...
1382    [Wait Essntl Sctt Mnrl Wtr, 2000, Plastic Bott...
1383    [Morr Svrs NAS Cola, 2000, Plastic Bottle, Col...
1384    [Waitrose Sctt M Wtr, 2000, Plastic Bottle, No...
Name: StockCode, Length: 1385, dtype: object

In [21]:
cats = pd.DataFrame(split_data.tolist(), columns=tokenizer.CATEGORIES)

In [22]:
cats

,Brand_First_Name,Pack_Size_ml,Pack_Type,Taste_Flavor_Fruits_Veg,Sugar_vs_Non_Sugar,Still_vs_Sparkling_for_Water,Segment
0,Pepsi Max S/Fr Cola,7920,Cans,Cola,Diet,Carbonated,Colas
1,Pepsi Max S/Fr Cola,2000,Plastic Bottle,Cola,Diet,Carbonated,Colas
2,Diet Coke,7920,Cans,Cola,Diet,Carbonated,Colas
3,Tesco Dbl Stng NAS Squash,15000,Plastic Bottle,Not Applicable,Diet,Non Carbonated,Squash
4,Coca Cola,7920,Cans,Cola,Regular,Carbonated,Colas
...,...,...,...,...,...,...,...
1380,Pure Life Sprng Mnrl Wtr,1500,Plastic Bottle,Non Flavoured,Regular,Non Carbonated,Unflavoured
1381,Old Jamaica Reg,330,Plastic Bottle,Other Fruit,Regular,Carbonated,Flavoured Carbonated
1382,Wait Essntl Sctt Mnrl Wtr,2000,Plastic Bottle,Non Flavoured,Regular,Non Carbonated,Unflavoured
1383,Morr Svrs NAS Cola,2000,Plastic Bottle,Cola,Diet,Carbonated,Colas


In [24]:
token_shares = pd.concat([tokenizer.token_shares, cats], axis=1)

In [25]:
token_shares

,StockCode,Quantity,Spend,share,Brand_First_Name,Pack_Size_ml,Pack_Type,Taste_Flavor_Fruits_Veg,Sugar_vs_Non_Sugar,Still_vs_Sparkling_for_Water,Segment
0,Pepsi Max S/Fr Cola|7920|Cans|Cola|Diet|Carbon...,20725,142486.739565,0.033501,Pepsi Max S/Fr Cola,7920,Cans,Cola,Diet,Carbonated,Colas
1,Pepsi Max S/Fr Cola|2000|Plastic Bottle|Cola|D...,61704,93836.382958,0.022063,Pepsi Max S/Fr Cola,2000,Plastic Bottle,Cola,Diet,Carbonated,Colas
2,Diet Coke|7920|Cans|Cola|Diet|Carbonated|Colas,11390,82349.189990,0.019362,Diet Coke,7920,Cans,Cola,Diet,Carbonated,Colas
3,Tesco Dbl Stng NAS Squash|15000|Plastic Bottle...,78033,78086.030010,0.018360,Tesco Dbl Stng NAS Squash,15000,Plastic Bottle,Not Applicable,Diet,Non Carbonated,Squash
4,Coca Cola|7920|Cans|Cola|Regular|Carbonated|Colas,7491,77949.630000,0.018327,Coca Cola,7920,Cans,Cola,Regular,Carbonated,Colas
...,...,...,...,...,...,...,...,...,...,...,...
1380,Pure Life Sprng Mnrl Wtr|1500|Plastic Bottle|N...,306,153.209999,0.000036,Pure Life Sprng Mnrl Wtr,1500,Plastic Bottle,Non Flavoured,Regular,Non Carbonated,Unflavoured
1381,Old Jamaica Reg|330|Plastic Bottle|Other Fruit...,375,142.469999,0.000033,Old Jamaica Reg,330,Plastic Bottle,Other Fruit,Regular,Carbonated,Flavoured Carbonated
1382,Wait Essntl Sctt Mnrl Wtr|2000|Plastic Bottle|...,278,142.320001,0.000033,Wait Essntl Sctt Mnrl Wtr,2000,Plastic Bottle,Non Flavoured,Regular,Non Carbonated,Unflavoured
1383,Morr Svrs NAS Cola|2000|Plastic Bottle|Cola|Di...,774,131.890001,0.000031,Morr Svrs NAS Cola,2000,Plastic Bottle,Cola,Diet,Carbonated,Colas


In [30]:
prods = pd.DataFrame(dset.StockCode, columns=['StockCode']).merge(token_shares, on='StockCode')

In [31]:
prods

,StockCode,Quantity,Spend,share,Brand_First_Name,Pack_Size_ml,Pack_Type,Taste_Flavor_Fruits_Veg,Sugar_vs_Non_Sugar,Still_vs_Sparkling_for_Water,Segment
0,Asda Mineral Water|6000|Plastic Bottle|Non Fla...,5736,9175.130129,0.002157,Asda Mineral Water,6000,Plastic Bottle,Non Flavoured,Regular,Non Carbonated,Unflavoured
1,Rubicon Exotic|1000|Cartons|Tropical/Exotic|Re...,1746,1938.290003,0.000456,Rubicon Exotic,1000,Cartons,Tropical/Exotic,Regular,Non Carbonated,Juice Drinks
2,Asda Fruit Juice|1000|Cartons|Orange|Regular|N...,22071,16286.719922,0.003829,Asda Fruit Juice,1000,Cartons,Orange,Regular,Non Carbonated,Pure Juices
3,Asda Lemonade|2000|Plastic Bottle|Lemonade Cle...,8952,2930.450061,0.000689,Asda Lemonade,2000,Plastic Bottle,Lemonade Clear,Regular,Carbonated,Lemonade
4,Asda Zero|2000|Plastic Bottle|Orange|Diet|Carb...,8169,3746.890004,0.000881,Asda Zero,2000,Plastic Bottle,Orange,Diet,Carbonated,Flavoured Carbonated
...,...,...,...,...,...,...,...,...,...,...,...
2005211,Copella Pr Jc|1350|Plastic Bottle|Other Multi ...,1405,3255.129998,0.000765,Copella Pr Jc,1350,Plastic Bottle,Other Multi Fruit,Regular,Non Carbonated,Pure Juices
2005212,Coca Cola|250|Cans|Cola|Regular|Carbonated|Colas,498,416.010007,0.000098,Coca Cola,250,Cans,Cola,Regular,Carbonated,Colas
2005213,Copella Pr Jc|900|Plastic Bottle|Apple|Regular...,507,852.599999,0.000200,Copella Pr Jc,900,Plastic Bottle,Apple,Regular,Non Carbonated,Pure Juices
2005214,Rbnsns NAS Frt Shoot|800|Plst Btl with Sports ...,2874,3049.619996,0.000717,Rbnsns NAS Frt Shoot,800,Plst Btl with Sports Cap,Other Multi Fruit,Diet,Non Carbonated,Juice Drinks


In [39]:
assert (dset.StockCode == prods.StockCode).all()

In [35]:
prods.StockCode

0          Asda Mineral Water|6000|Plastic Bottle|Non Fla...
1          Rubicon Exotic|1000|Cartons|Tropical/Exotic|Re...
2          Asda Fruit Juice|1000|Cartons|Orange|Regular|N...
3          Asda Lemonade|2000|Plastic Bottle|Lemonade Cle...
4          Asda Zero|2000|Plastic Bottle|Orange|Diet|Carb...
                                 ...                        
2005211    Copella Pr Jc|1350|Plastic Bottle|Other Multi ...
2005212     Coca Cola|250|Cans|Cola|Regular|Carbonated|Colas
2005213    Copella Pr Jc|900|Plastic Bottle|Apple|Regular...
2005214    Rbnsns NAS Frt Shoot|800|Plst Btl with Sports ...
2005215    Tesco Pr Jc|1000|Plastic Bottle|Orange|Regular...
Name: StockCode, Length: 2005216, dtype: object